In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import resnet_model
import os

%matplotlib inline

/home/barney_awesome_li/anaconda3/envs/tfgpu/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [37]:
# Load training data set
train = pd.read_json('../data/train.json')

In [38]:
order_num = np.random.rand(train.shape[0])
train['order'] = order_num
train.sort_values('order', axis=0, inplace=True)
print(train.head())

                                                band_1  \
456  [-19.818336, -19.40139, -19.266779, -21.039583...   
916  [-24.378555, -22.966972, -28.159756, -29.9028,...   
38   [-25.560505, -25.560505, -27.303579, -22.67576...   
569  [-29.871798, -27.842697, -27.842752, -25.07675...   
808  [-24.384838, -22.397192, -22.202042, -23.67067...   

                                                band_2        id inc_angle  \
456  [-24.643572, -26.277029, -26.277084, -29.08071...  e7bedace   38.9318   
916  [-28.987532, -29.4331, -26.708744, -26.708782,...  28224273        na   
38   [-29.910183, -29.910183, -31.848454, -29.08247...  4cfca6da   34.4721   
569  [-28.598419, -26.503763, -26.199017, -31.92520...  f3e78568   39.6912   
808  [-30.657219, -28.821859, -25.163292, -25.43910...  1b059732   41.8181   

     is_iceberg     order  
456           0  0.000146  
916           0  0.000567  
38            1  0.000940  
569           0  0.001487  
808           1  0.001866  


In [39]:
raw_y = train['is_iceberg'].tolist()
tensor_y = tf.one_hot(raw_y, depth=2)
with tf.Session() as sess:
    total_y = np.array(sess.run(tensor_y))
print(total_y.shape)

(1604, 2)


In [40]:
np.array(train['is_iceberg'].tolist()).reshape(-1, 1)

array([[0],
       [0],
       [1],
       ..., 
       [0],
       [1],
       [1]])

In [41]:
is_iceberg = np.array(train['is_iceberg'].tolist()).reshape(-1, 1)
raw_x_1 = np.array(train['band_1'].tolist())
raw_x_2 = np.array(train['band_2'].tolist())
total_x = np.concatenate((is_iceberg, raw_x_1, raw_x_2), axis = 1)
#total_x = raw_x_1
print(total_x.shape)

(1604, 11251)


In [42]:
dev_x = total_x[0:300,:]
dev_y = total_y[0:300,:]
print(dev_x.shape)
train_x = total_x[300:,:]
train_y = total_y[300:,:]
print(train_x.shape)
print(train_y.shape)

(300, 11251)
(1304, 11251)
(1304, 2)


In [55]:
class MiniBatch(object):
    def __init__(self, x, y, batch_size):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.generator = self.slice_random(self.x, self.y, self.batch_size)
    
    def next_batch(self):
        try:
            return(next(self.generator))
        except StopIteration as e:
            self.generator = self.slice_random(self.x, self.y, self.batch_size)
            return(next(self.generator))
        
    @staticmethod    
    def slice_random(X, Y, mini_batch_size):
        tmp_X = X.copy()
        tmp_Y = Y.copy()
        cur_index = 0
        while(tmp_X.shape[0]>0):
            pick_size = tmp_X.shape[0] if tmp_X.shape[0]<mini_batch_size else mini_batch_size
            mini_batch_x = tmp_X[cur_index:cur_index + pick_size,:]
            mini_batch_y = tmp_Y[cur_index:cur_index + pick_size,:]
            cur_index += pick_size
            yield mini_batch_x, mini_batch_y

In [56]:
test_mini_batch_gen = MiniBatch(train_x, train_y, 10)

In [48]:
test_mini_batch_x, test_mini_batch_y = test_mini_batch_gen.next_batch()
print(test_mini_batch_x[:,0])

[ 0.  1.  0.  0.  0.]


In [57]:
for i in range(200):
    test_mini_batch_x, test_mini_batch_y = test_mini_batch_gen.next_batch()
    print(test_mini_batch_x[:,0], " \n", test_mini_batch_y)

[ 0.  1.  0.  0.  0.  0.  1.  1.  0.  0.]  
 [[ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
[ 1.  0.  0.  1.  0.  0.  1.  1.  0.  1.]  
 [[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]]
[ 1.  1.  0.  1.  1.  0.  0.  0.  1.  0.]  
 [[ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]]
[ 1.  0.  1.  0.  0.  1.  1.  1.  0.  0.]  
 [[ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]]
[ 0.  0.  0.  0.  0.  1.  0.  1.  1.  1.]  
 [[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]
[ 0.  0.  0.  1.  1.  0.  1.  0.  1.  1.]  
 [[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]]
[ 1.  1.  1.  0.  0.  1.  1.  1.  0.  0.]  
 [[ 0.  1.]
 [ 0.  1